## Rank the surfaces based on the screening readout

#### Setup

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
import PIL
import os
from rankFeaturesFunctions import calculateRank,plotSurfaceRankMeasurements,makePairPlot,filterRank,plotRank,boxPlotTopAndBottom,sortRanked,showImagesFlat,showImagesTopAndBottom
from scipy.stats import ks_2samp, anderson_ksamp

In [11]:
pip install tables

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.6 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 86.5 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.5/475.5 KB 79.2 MB/s eta 0:00:00
You should consider upgrading via the '/sw/rl8/zen/app/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


#### Step 1: Load the data objects

In [7]:
#pathToImageObject=os.getcwd()+"/DataAnalysis/"       
locationOfImageObject="cleaned_rawImagedataFI_final.csv"
# check if file exists
#if os.path.isfile(locationOfImageObject):
imagedata=pd.read_csv(locationOfImageObject,low_memory=False)
    # replace any empty strings with NaNs
imagedata=imagedata.replace('', np.nan)
#else:
#    print("File not found, check if you finished outlier removal")


In [12]:
# Define ontology mappings (adjust as needed)
ontology_mappings = {
    'CL_cell_type': 'CL:0000653',                  # Podocyte
    'OBI_assay_type': 'OBI:0000185',               # Imaging assay
    'OBI_microscopy_method': 'OBI:0000824',        # Fluorescence microscopy
    'OBI_staining_protocol': 'OBI:0000187',
    'EDAM_file_format': 'EDAM:data_2966',          # ND2 file format
    'EDAM_segmentation_method': 'EDAM:operation_3745',  # CellProfiler
    'EDAM_morphological_analysis': 'EDAM:operation_3460'
}

# Add ontology columns to your dataframe
for col_name, ontology_id in ontology_mappings.items():
    imagedata[col_name] = ontology_id

# Save updated dataframe
imagedata.to_hdf('Whole_screen_podocyte_database_with_ontologies.h5', key='imagedata', index=False)

print("Ontology columns added successfully!")

Ontology columns added successfully!


In [13]:
imagedata

AreaOccupied_AreaOccupied_CellBody_new  \
0                                    195441.0   
1                                    190993.0   
2                                    157466.0   
3                                    128276.0   
4                                    159374.0   
...                                       ...   
31915                                146459.0   
31916                                113561.0   
31917                                136466.0   
31918                                108352.0   
31919                                132796.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_new  \
0                                           41149.0   
1                                           34146.0   
2                                           21593.0   
3                                           30686.0   
4                                           36365.0   
...                                             ...   
31915                                       28944.0   
31916                                       21965.0   
31917                                       22229.0   
31918                                       18080.0   
31919                                       35925.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_target  \
0                                              38157.0   
1                                              33484.0   
2                                              19752.0   
3                                              28584.0   
4                                              35031.0   
...                                                ...   
31915                                          24587.0   
31916                                          19472.0   
31917                                          18386.0   
31918                                          12223.0   
31919                                          27669.0   

       AreaOccupied_AreaOccupied_MaskedNuclei  \
0                                    147590.0   
1                                    178264.0   
2                                    112684.0   
3                                    115514.0   
4                                    147877.0   
...                                       ...   
31915                                128723.0   
31916                                 96719.0   
31917                                109961.0   
31918                                 73657.0   
31919                                106458.0   

       AreaOccupied_AreaOccupied_Nephrin  AreaOccupied_Perimeter_CellBody_new  \
0                               166158.0                              17129.0   
1                               210194.0                              18661.0   
2                               133496.0                              13267.0   
3                               138323.0                              11218.0   
4                               200466.0                              16275.0   
...                                  ...                                  ...   
31915                           165853.0                              13291.0   
31916                           153661.0                              10378.0   
31917                           143216.0                              12602.0   
31918                           146310.0                              10716.0   
31919                           133671.0                              10817.0   

       AreaOccupied_Perimeter_FilteredNuclei_new  \
0                                         6242.0   
1                                         5837.0   
2                                         3797.0   
3                                         4632.0   
4                                         5380.0   
...                                          ...   
31915                                     4030.0   
31916                                     3448.0   
31917                         

#### Step 2: Rank the data based on feature of interest

##### Step 2.1: Select the feature of interest

In [42]:
## write an console to select the feature of interest
featureOfInterestUser="Mean_FilteredNuclei_new_AreaShape_FormFactor"

##### Step 2.2: rank the surfaces to the feature of interest

In [43]:
imagedata['FeatureIdx'].value_counts()

2177    26
1264    18
1616    18
569     18
1933    18
        ..
1659    10
201     10
234      9
271      9
75       8
Name: FeatureIdx, Length: 2177, dtype: int64

In [44]:
# imagedata_and_counts = imagedata.merge(imagedata['FeatureIdx'].value_counts(),left_on='FeatureIdx', right_index=True)
# imagedata = imagedata[imagedata_and_counts.FeatureIdx_y > 1]

In [45]:
imagedata

AreaOccupied_AreaOccupied_CellBody_new  \
0                                    195441.0   
1                                    190993.0   
2                                    157466.0   
3                                    128276.0   
4                                    159374.0   
...                                       ...   
31915                                146459.0   
31916                                113561.0   
31917                                136466.0   
31918                                108352.0   
31919                                132796.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_new  \
0                                           41149.0   
1                                           34146.0   
2                                           21593.0   
3                                           30686.0   
4                                           36365.0   
...                                             ...   
31915                                       28944.0   
31916                                       21965.0   
31917                                       22229.0   
31918                                       18080.0   
31919                                       35925.0   

       AreaOccupied_AreaOccupied_FilteredNuclei_target  \
0                                              38157.0   
1                                              33484.0   
2                                              19752.0   
3                                              28584.0   
4                                              35031.0   
...                                                ...   
31915                                          24587.0   
31916                                          19472.0   
31917                                          18386.0   
31918                                          12223.0   
31919                                          27669.0   

       AreaOccupied_AreaOccupied_MaskedNuclei  \
0                                    147590.0   
1                                    178264.0   
2                                    112684.0   
3                                    115514.0   
4                                    147877.0   
...                                       ...   
31915                                128723.0   
31916                                 96719.0   
31917                                109961.0   
31918                                 73657.0   
31919                                106458.0   

       AreaOccupied_AreaOccupied_Nephrin  AreaOccupied_Perimeter_CellBody_new  \
0                               166158.0                              17129.0   
1                               210194.0                              18661.0   
2                               133496.0                              13267.0   
3                               138323.0                              11218.0   
4                               200466.0                              16275.0   
...                                  ...                                  ...   
31915                           165853.0                              13291.0   
31916                           153661.0                              10378.0   
31917                           143216.0                              12602.0   
31918                           146310.0                              10716.0   
31919                           133671.0                              10817.0   

       AreaOccupied_Perimeter_FilteredNuclei_new  \
0                                         6242.0   
1                                         5837.0   
2                                         3797.0   
3                                         4632.0   
4                                         5380.0   
...                                          ...   
31915                                     4030.0   
31916                                     3448.0   
31917                         

In [46]:
surfacesRank=calculateRank(ScreenData=imagedata,
                           featureOfInterest=featureOfInterestUser)

/bme001.mnt/home/20212358/rankFeaturesFunctions.py:26: UserWarning: p-value capped: true value larger than 0.25
  _,_,pvalue=anderson_ksamp([sg1, sg2])


##### Step 2.3: Plot the scoring metrics of the ranked surfaces

In [47]:
surfacesRank.max()

FeatureIdx                                                      2177.000000
Screen_ADpvalue_Mean_FilteredNuclei_new_AreaShape_FormFactor       0.250000
Screen_KSpvalue_Mean_FilteredNuclei_new_AreaShape_FormFactor       1.000000
Screen_SNR_Mean_FilteredNuclei_new_AreaShape_FormFactor           58.845446
Screen_Mean_Mean_FilteredNuclei_new_AreaShape_FormFactor           0.744939
Screen_SD_Mean_FilteredNuclei_new_AreaShape_FormFactor             0.126169
Screen_Median_Mean_FilteredNuclei_new_AreaShape_FormFactor         0.748509
dtype: float64

In [9]:
plotSurfaceRankMeasurements(surfacesRank)

##### Step 2.4: Display the relation between the different scoring metrics

In [10]:
makePairPlot(surfacesRank)

#### Step 3: Filter the ranked data based on a feature of interest

Perfom a rank based filtering based on the p-value and the Signal to Noise Ratio (SNR)

step 3.1: filter the data based on the p-values, set the lower and upper bound of p-values to remove (default lower bound: 0.05 and upper bound:1)

In [48]:
# Lower and upper bound are the p-values not to select (so remove data if p-value>0.05)
metricOfInterestToFilter="Screen_ADpvalue_Mean_FilteredNuclei_new_AreaShape_FormFactor"
lowerbound=0.05
upperbound=1

In [49]:
surfacesRankPVF=filterRank(rank=surfacesRank,
           filterMetric=metricOfInterestToFilter,
           valuesRangeToInclude=[lowerbound,upperbound])

101 unique surafces were excluded from the analysis
The following FeatureIdx were excluded based on the Screen_ADpvalue_Mean_FilteredNuclei_new_AreaShape_FormFactor
5
23
31
45
62
66
98
103
113
138
146
185
204
208
225
226
231
237
259
290
307
318
332
379
383
392
419
428
451
452
492
493
580
593
602
618
619
625
680
729
752
758
763
778
790
797
810
863
879
915
925
932
943
953
964
988
1060
1074
1087
1096
1115
1142
1154
1168
1173
1231
1249
1285
1297
1357
1366
1400
1427
1430
1435
1436
1448
1455
1520
1526
1608
1656
1658
1685
1702
1763
1777
1786
1813
1844
1847
1853
1878
1887
1889
1952
1973
2024
2080
2146
2162


step 3.2: filter the data based on the SNR ration, set the lower and upper bound of SNR values (default lower bound:1 and upper bound:20)

In [50]:
featureOfInterestToFilter2="Screen_SNR_Mean_FilteredNuclei_new_AreaShape_FormFactor"
lowerbound=2
upperbound=20

In [51]:
surfacesRankPVF_SNR=filterRank(rank=surfacesRankPVF,
           filterMetric=featureOfInterestToFilter2,
           valuesRangeToInclude=[lowerbound,upperbound])

284 unique surafces were excluded from the analysis
The following FeatureIdx were excluded based on the Screen_SNR_Mean_FilteredNuclei_new_AreaShape_FormFactor
21
34
38
39
48
51
68
72
76
81
99
124
136
137
145
160
163
166
171
193
194
222
229
250
298
337
462
515
526
530
543
547
550
562
576
578
587
610
611
623
633
642
652
660
661
662
664
675
676
679
684
686
687
705
710
714
722
723
727
736
740
743
745
747
751
754
770
771
772
774
776
781
785
787
791
793
798
799
802
805
806
807
811
814
815
818
822
826
832
833
842
843
844
849
850
867
876
881
882
893
900
906
912
917
918
924
935
942
951
957
962
971
973
977
987
1001
1005
1006
1024
1029
1036
1039
1041
1049
1055
1057
1058
1062
1067
1069
1072
1076
1077
1084
1088
1089
1090
1097
1105
1106
1121
1136
1150
1153
1157
1158
1171
1172
1176
1178
1182
1192
1194
1195
1206
1214
1216
1225
1233
1235
1236
1242
1256
1257
1272
1274
1277
1282
1291
1292
1296
1300
1317
1330
1343
1344
1360
1373
1374
1379
1402
1497
1549
1619
1641
1643
1644
1669
1694
1714
1719
1720
1745
1

Step 3.4: Check how relations between ranking metrics changed after applying filtering steps

In [15]:
makePairPlot(surfacesRankPVF_SNR)

#### Step 4: Plot the ranking results

Step 4.1: Plot the rank versus the feature of interest (with the flat surface visualized by the large dot)

In [37]:
plotRank(rank=surfacesRankPVF_SNR, 
         featureOfInterest="Screen_Mean_Mean_FilteredNuclei_new_AreaShape_Area")

Step 4.2: Plot boxplots of the top and botom scoring surfaces

In [38]:
boxPlotTopAndBottom(screenData=imagedata,
                 rank=surfacesRankPVF_SNR,
                 featureOfInterest='Screen_Mean_Mean_FilteredNuclei_new_AreaShape_Area',
                numberOfSurfaces=25)

/bme001.mnt/home/20212358/rankFeaturesFunctions.py:126: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(labels)


In [18]:
#showImagesFlat(ScreenData=rawImagedataFI_o2, 
#               featureOfInterest="Median_Nuclei_Intensity_IntegratedIntensity_CorrYap", 
#               stainingFile="FileName_actin", 
#               mode="raw" )

In [19]:
#showImagesTopAndBottom(rank=surfacesRankPVF_SNR, 
#                       ScreenData=rawImagedataFI_o2, 
#                       featureOfInterest="Screen_Mean_Median_Nuclei_Intensity_IntegratedIntensity_CorrYap",
#                       numberOfSurfaces=5, 
#                           stainingFile="FileName_Yap",
#                       mode="raw",
#                       surfPerRow=6)

#### Step 5: Perform a final ranking based on the feature of interest and save the data

In [52]:
FeatureOfInterestFinal="Screen_Mean_Mean_FilteredNuclei_new_AreaShape_FormFactor"

In [53]:
surfacesRankPVF_SNR_sorted=sortRanked(rank=surfacesRankPVF_SNR, 
         featureOfInterest=FeatureOfInterestFinal)

In [54]:
surfacesRankPVF_SNR_sorted.to_csv(os.getcwd()+"/DataAnalysis_withJan/Screen_Mean_Mean_FilteredNuclei_new_AreaShape_FormFactor.csv", index=False)